在`Ruby`中，类的定义有所不同，使用`class`关键字时。你不仅仅在规定对象的行为方式，你实际上在运行代码。

类只不过是一个增强的模块；因此本章学到的所有只是也都可以应用于模块。

你可能认为定义类就是定义方法。其实，你可以在类定义中放入任何代码

In [1]:
def MyClass
  puts 'Hello'
end

:MyClass

In [2]:
result = class MyClass
  self
end

MyClass

In [3]:
def testt
end

:testt

在程序的顶层，当前类是`Object`，这是`main`对象所属的类（这就是你在顶层定义方法会称为`Object`实例方法的原因。）

In [4]:
p self.class

Object


Object

In [5]:
p self

main


main

In [6]:
self.class

Object

In [7]:
self.private_methods

[:include, :using, :public, :private, :define_method, :DelegateClass, :Digest, :MyClass, :testt, :sprintf, :format, :Integer, :Float, :String, :Array, :Hash, :fail, :iterator?, :__method__, :catch, :__dir__, :loop, :global_variables, :throw, :block_given?, :raise, :binding, :URI, :__callee__, :eval, :trace_var, :untrace_var, :at_exit, :Rational, :Complex, :select, :set_trace_func, :gem, :caller, :caller_locations, :`, :test, :fork, :exit, :Pathname, :sleep, :respond_to_missing?, :load, :gem_original_require, :syscall, :open, :printf, :print, :putc, :puts, :gets, :readlines, :readline, :initialize_copy, :initialize_clone, :initialize_dup, :system, :p, :exec, :exit!, :proc, :lambda, :srand, :rand, :spawn, :abort, :pp, :trap, :require, :require_relative, :autoload, :autoload?, :local_variables, :warn, :method_missing, :singleton_method_added, :singleton_method_removed, :singleton_method_undefined, :initialize]

In [8]:
class C
  def m1
    def m2; end
  end
end

class D < C; end

obj = D.new
obj.m1

C.instance_methods(false)

[:m1, :m2]

在一个方法中，当前类就是当前对象的类。（试着在一个方法中用`def`关键字定义另外一个方法，你会发现这个新方法会定义在`self`所属的类中）

`Module#class_eval`方法会在一个已存在类的上下文中执行一个块

In [9]:
def add_method_to(a_class)
  a_class.class_eval do
    def m; 'Hello!';end
  end
end

add_method_to String
"abc".m

"Hello!"

`Module#class_eval`方法和`Object#instance_eval`方法截然不同。`instance_eval`方法只修改`self`，而`class_eval`方法会同时修改`self`和当前类

`Module#class_eval`方法实际上比`class`关键字更加灵活。可以对任何代表类的变量使用`class_eval`方法，而`class`关键字只能使用常量。另外,`class`关键字会打开一个新的作用域，这样将丧失当前绑定的可见性。而`class_eval`方法则使用**扁平作用域**

`instance_eval`和`class_eval`方法该如何选择呢？通常我们用`instance_eval`打开非类的对象；而用`class_eval`打开类定义，然后用`def`定义方法。如果要打开的对象也是类，那该用`instance_eval`还是`class_eval`呢？

如果你只想修改`self`，那么`instance_eval`方法和`class_eval`方法都可以出色的完成任务。不过，你应该选择更能准确表达你意图的方法。如果你希望打开一个对象，但并不关心它是不是一个类，那么`instance_eval`就很很好。如果你想使用打开类技巧来修改类，那么`class_eval`方法显然是更好的选择。

Ruby解释器假定所有的实例变量都属于当前对象`self`。在类定义时也如此。

In [11]:
class MyClass
  @my_val = 1
end

1

In [13]:
class MyClass
  @my_val = 1
  def self.read; @my_var; end
  def write; @my_var = 2; end
  def read; @my_var; end
end

:read

In [16]:
obj = MyClass.new
p obj.read
p obj.write
p obj.read
p MyClass.read

nil
2
2
nil


上面的程序定义了两个实例变量，它们正好都叫`@my_var`，但是它们分属不同的作用域。要清楚怎么回事，需要牢记类也是对象。

## 类变量

In [22]:
class C
  @@v = 1
end

1

In [23]:
class D < C
  def my_method; @@v; end
end

:my_method

类变量和类实例变量不同，它们可以被子类或者类的实例所使用。

In [25]:
D.new.my_method

1

不幸的是，类变量有一个不好的怪癖，下面是一个例子：

In [27]:
@@v = 1

class MyClass
  @@v = 2
end

@@v

(pry):90: warning: class variable access from toplevel
(pry):96: warning: class variable access from toplevel


2

得到这样的结果是因为类变量并不真正属于类，它属于类体系结构。由于`@@v`定义于`main`的上下文，它属于`main`的类`Object`，所以也属于`Object`所有的后代。

In [29]:
c = Class.new(Array) do
  def my_method
    'Hello!'
  end
end

#<Class:0x1b80640>

In [32]:
c.name

In [35]:
MyClass = c
MyClass.name

(pry):114: warning: already initialized constant MyClass
(pry):112: warning: previous definition of MyClass was here


"MyClass"

有趣的是，`Ruby`在这里耍了一个小把戏。当你把一个匿名类赋值给一个变量时，`Ruby`知道你是想给这个类命名，他会对这个类说，这是你的新名字